## 实验题目5 高斯(Gauss)列主元消去法

### 代码实现

In [99]:
using Printf
using LinearAlgebra

In [100]:
# from: https://stackoverflow.com/questions/58667332/is-there-a-way-to-swap-columns-in-o1-in-julia
function swapcols!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 1)
        X[k, i], X[k, j] = X[k, j], X[k, i]
    end
end
# from: https://discourse.julialang.org/t/swap-cols-rows-of-a-matrix/47904/9
function _swapcol!(x, i, j)
    for k in axes(x, 1)  # <- give dimension as input to axes function
        x[k, i], x[k, j] = x[k, j], x[k, i]
    end
end

_swapcol! (generic function with 1 method)

In [101]:
function swaprows!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 2)
        X[i, k], X[j, k] = X[j, k], X[i, k]
    end
end

swaprows! (generic function with 1 method)

In [102]:
# https://stackoverflow.com/questions/45396685/what-does-an-exclamation-mark-mean-after-the-name-of-a-function
# https://people.richland.edu/james/lecture/m116/matrices/pivot.html
function pivoting!(A::Matrix{Float64}, k::Integer, n::Integer)
    val, idx = findmax(A[k:n, k])
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end
function pivoting!(A::Matrix{Float64}, b::Vector{Float64}, k::Integer, n::Integer, implicit::Bool)
    s = [maximum(A[i, k:n]) for i in k:n]
    if 0 in s
        println("Cannot solve a singular matrix!")
        return
    end
    if implicit
        val, idx = findmax(A[k:n, k] ./ s[1:n-k+1])
    else
        A[k:n, k:n] = A[k:n, k:n] ./ s
        b[k:n] = b[k:n] ./ s
        val, idx = findmax(A[k:n, k])
    end
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end

pivoting! (generic function with 4 methods)

In [103]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64})
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, k, n)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

In [104]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64}, implicit::Bool)
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, b, k, n, implicit)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

### Random Test 1

In [106]:
# test random result of standard library 
# test pass
for i in 1:100
    A = rand(4, 4)
    b = rand(4)
    # display(norm(A \ b - gauss(size(A, 1), A, b), 2))
    # display(norm(A \ b - gauss(size(A, 1), A, b,true), 2))
    display(norm(A \ b - gauss(size(A, 1), A, b, Null), 2))
    # display(norm(A \ b - gauss_explicit(size(A, 1), A, b),2))
    # display(norm(A \ b - gauss_implicit(size(A, 1), A, b),2))
    # display([A \ b  gauss_explicit(size(A, 1), A, b)])

end

# A = [0.1 0.1; 2.3 2.3]
# b = [1.0, 3]
# try
#     A \ b
# catch SigularException
#     println("Cannot solve a singular matrix!")
# end

LoadError: UndefVarError: Null not defined